<a href="https://colab.research.google.com/github/Mohit1052/Face-Mask-Detection/blob/main/Face_Mask_Detection_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
from keras.models import Sequential
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator,load_img
from keras.layers import Dense, Flatten,Conv2D,Dropout,SeparableConv2D,MaxPooling2D
import matplotlib.pyplot as plt
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.models import load_model
from sklearn.metrics import classification_report
import numpy as np

In [ ]:
my_new_model = Sequential()
my_new_model.add(Conv2D(32,(7,7),activation='relu',input_shape=(224,224,3)))
my_new_model.add(MaxPooling2D((2,2)))
my_new_model.add(Dropout(0.2))
my_new_model.add(Conv2D(64,(5,5),activation='relu'))
my_new_model.add(MaxPooling2D((2,2)))
my_new_model.add(Dropout(0.2))

my_new_model.add(Conv2D(32,(3,3),activation='relu'))
my_new_model.add(MaxPooling2D((2,2)))
my_new_model.add(Dropout(0.3))

my_new_model.add(Conv2D(16,(3,3),activation='relu'))
my_new_model.add(MaxPooling2D((2,2)))
my_new_model.add(Dropout(0.3))

my_new_model.add(Flatten())
my_new_model.add(Dense(16,activation='relu'))
my_new_model.add(Dropout(0.5))
my_new_model.add(Dense(2, activation='softmax'))
# Indicate whether the first layer should be trained/changed or not.
my_new_model.layers[0].trainable = False  

my_new_model.compile(optimizer='adam', 
                     loss='categorical_crossentropy', 
                     metrics=['accuracy'])

In [ ]:
my_new_model.summary()

In [ ]:
image_size = 224
data_generator = ImageDataGenerator(featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    samplewise_center=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    brightness_range=None,
    shear_range=0.0,
    zoom_range=0.0,
    channel_shift_range=0.0,
    fill_mode="nearest",
    cval=0.0,
    vertical_flip=False,
    rescale=1.0/255.0,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.2,
    dtype=None)

train_generator = data_generator.flow_from_directory(
                                        directory=r"/content/gdrive/MyDrive/dataset1",
                                        target_size=(image_size, image_size),
                                        subset='training',
                                        batch_size=10,
                                        shuffle=True,
                                        class_mode='categorical')
validation_generator = data_generator.flow_from_directory(
                                        directory=r"/content/gdrive/MyDrive/dataset1",
                                        target_size=(image_size, image_size),
                                        subset='validation',
                                        shuffle=True,
                                        batch_size=10,
                                        class_mode='categorical')

classes = ['without_mask', 'with_mask']


In [ ]:
fit_stats = my_new_model.fit_generator(generator = train_generator,
                                steps_per_epoch = len(train_generator),
                                epochs =20,
                                validation_data = validation_generator,
                                validation_steps=1)
 

In [ ]:
plt.plot(fit_stats.history['loss'],'g^',label='Training Loss')
plt.plot(fit_stats.history['val_loss'],'r--',label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('loss')
plt.grid(True)
plt.legend()
plt.show()
plt.plot(fit_stats.history['accuracy'],'go',label='Training Accuracy')
plt.plot(fit_stats.history['val_accuracy'],'r--',label='Validation Accuracy')
plt.title('Training vs Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.grid(True)
plt.legend()
plt.show()




In [ ]:
lab = validation_generator.classes
pred= my_new_model.predict(validation_generator)

predicted_class_indices=np.argmax(pred,axis=1)

labels = (validation_generator.class_indices)
labels2 = dict((v,k) for k,v in labels.items())
predictions = [labels2[k] for k in predicted_class_indices]

In [ ]:
lab

In [ ]:
lab.shape

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, mean_absolute_error, precision_score, recall_score

rc = roc_curve(predicted_class_indices,lab)
cf_matrix = confusion_matrix(predicted_class_indices,lab)
cf_report = classification_report(predicted_class_indices,lab)
print('Confusion matrix report of the model : \n{}'.format(cf_matrix))

In [ ]:
print('Classification report of the model : \n{}'.format(cf_report))

In [ ]:
class_names = classes
import itertools

def plot_confusion_matrix(cf_matrix, classes, title='Confusion matrix', cmap=plt.cm.Blues,normalize=True):
    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(10, 10))
    plt.grid(b=False)
    plt.imshow(cf_matrix, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if class_names is not None:
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)
    if normalize:
        cf_matrix = cf_matrix.astype('float') / cf_matrix.sum(axis=1)[:, np.newaxis]
    
    thresh = cf_matrix.max() / 1.5 if normalize else cf_matrix.max() / 2
    for i, j in itertools.product(range(cf_matrix.shape[0]), range(cf_matrix.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cf_matrix[i, j]),
                     horizontalalignment="center",
                     color="black" if cf_matrix[i, j] > thresh else "white")
        else:
            plt.text(j, i, "{:,}".format(cf_matrix[i, j]),
                     horizontalalignment="center",
                     color="black" if cf_matrix[i, j] > thresh else "white")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()
# compute confusion matrix
cnf_matrix = confusion_matrix(predicted_class_indices,lab)
np.set_printoptions(precision=np.random.randint(0,9))

# plot confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=False ,title='Confusion matrix',cmap='Reds')
plt.show()

In [ ]:
my_new_model.save_weights('model_wieghts.h5')
my_new_model.save('Face_detection_model.h5')

In [ ]:
import cv2
import keras
from google.colab.patches import cv2_imshow
import keras
from keras.models import load_model
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform

with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
        model = load_model("/content/Face_detection_model.h5")

In [ ]:
model.summary()

In [ ]:
image=cv2.imread('/content/gdrive/MyDrive/dataset1/with_mask/104-with-mask.jpg')

In [ ]:
prototxtPath = '/content/gdrive/MyDrive/face_detector/deploy.prototxt'
weightsPath = '/content/gdrive/MyDrive/face_detector/res10_300x300_ssd_iter_140000.caffemodel'
face_model = cv2.dnn.readNet(prototxtPath, weightsPath)

In [ ]:
height, width = image.shape[:2]
blob = cv2.dnn.blobFromImage(image, 1.0, (224, 224), (104.0, 177.0, 123.0))
face_model.setInput(blob)
detections=face_model.forward()

In [ ]:
from google.colab.patches import cv2_imshow
threshold = 0.2
person_with_mask = 0
person_without_mask = 0
for i in range(0, detections.shape[2]):
    confidence = detections[0, 0, i, 2]
    if confidence > 0.5:
        box = detections[0, 0, i, 3:7] * np.array([width, height, width, height])
        (startX, startY, endX, endY) = box.astype("int")
        (startX, startY) = (max(0, startX), max(0, startY))
        (endX, endY) = (min(width - 1, endX), min(height - 1, endY))

        face = image[startY:endY, startX:endX]
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face = cv2.resize(face, (224, 224))
        face = img_to_array(face)
        face = preprocess_input(face)
        face = np.expand_dims(face, axis=0)

        (mask, withoutMask) = model.predict(face)[0]
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

        cv2.putText(image, label, (startX, startY - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)
cv2_imshow(image)

In [ ]:
|